In [ ]:
import os
from flask import Flask, render_template, request, session, request, redirect, Response
from flask import url_for
import cv2
import uuid
import base64
import json
from utils  import *
import time
import subprocess as sp
import traceback
import numpy as np
import pandas

FPS = 30 
SKIP = 0 
WFPS = 25


app = Flask(__name__)

WASABI = Wasabi()

if not WASABI.bucket_exists("tf-fd-demo"):
    WASABI.s3.create_bucket(Bucket="tf-fd-demo")
    WASABI.set_bucket_policy("tf-fd-demo")
    
base_url = "https://s3.wasabisys.com/{}/".format("tf-fd-demo")

@app.route('/process-video', methods=["POST"])
def process_url():
    
    payload = request.get_json(force=True)
    uid = uuid.uuid4()

    with open("./%s.webm" % uid, "wb") as f:
        video = base64.b64decode(payload["video"].split(",")[1])
        f.write(video)
    
    cap = cv2.VideoCapture("./%s.webm" % uid)
    out = cv2.VideoWriter(
        './%s.mp4' % uid, 
        cv2.VideoWriter_fourcc(*'mp4v'), int(WFPS), 
        (int(cap.get(3)), int(cap.get(4))), True)

    counter = 0
    while True:
        ret, frame = cap.read()
        try:
            if not ret:
                break
        except Exception as e:
            print(e)
            print(traceback.format_exc())

        out.write(frame)
        counter += 1
    cap.release()
    out.release()

    encode_segement = [
        "ffmpeg",
        "-re", "-i", "%s.mp4" % uid,
        "-g", "52",
        "-acodec", "copy",
        "-vcodec", "libx264", 
        "-crf", "27",
        "-preset", "ultrafast",
        "-c:a", "copy",
        "-f", "mp4",
        "%s_tmp.mp4" % uid
    ]
    devnull = open(os.devnull, 'w')
    sp.call(encode_segement, stdout=devnull, stderr=devnull)
    fragment_segement = [
        "./mp4fragment","%s_tmp.mp4" % uid, "%s_out.mp4" % uid
    ]
    sp.call(fragment_segement, stdout=devnull, stderr=devnull)

    out = None
    WASABI.upload_file("%s_out.mp4" % uid, "tf-fd-demo", "%s_out.mp4" % uid)
    
    url = base_url + "%s_out.mp4" % uid

    os.system("rm %s.webm" % uid)
    os.system("rm %s.mp4" % uid)
    os.system("rm %s_tmp.mp4" % uid)
    os.system("rm %s_out.mp4" % uid)
    return json.dumps({"success":True, "URL": url}), 200, {"Content-Type":"application/json"}


@app.route('/', defaults={'path': ''})
@app.route('/<path:path>')
def home(path):
    if path == '':
        page = 'index.html'
    else:
        page = path
    return render_template(page)


if __name__ == '__main__':
    app.run(debug=False, host= '0.0.0.0', threaded=False, port=5001)